In [97]:
from binance.spot import Spot
from datetime import datetime
client = Spot()

In [127]:
def get_ts_for_days():
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2024, 1, 2)
    day_count = (end_date - start_date).days + 1
    for  n in range(day_count):
        temp_date = start_date + timedelta(days = n)
        #print(temp_date.timestamp())
        yield temp_date, int(temp_date.timestamp())


In [131]:
for date, timestamp in get_ts_for_days():
    print(type(timestamp))
    print(timestamp)
    klines = client.klines("BTCUSDT", "2h",startTime = timestamp, limit = 12)
    print(len(klines))
    break

<class 'float'>
1577833200.0
12


In [132]:
klines

[[1502942400000,
  '4261.48000000',
  '4328.69000000',
  '4261.32000000',
  '4315.32000000',
  '70.41592500',
  1502949599999,
  '302670.96196053',
  273,
  '56.60857400',
  '243560.75767140',
  '0'],
 [1502949600000,
  '4330.29000000',
  '4349.99000000',
  '4287.41000000',
  '4349.99000000',
  '11.67294000',
  1502956799999,
  '50523.37096975',
  61,
  '7.40515300',
  '32086.66423939',
  '0'],
 [1502956800000,
  '4333.32000000',
  '4445.78000000',
  '4333.32000000',
  '4444.00000000',
  '11.73643000',
  1502963999999,
  '51458.85952932',
  71,
  '11.07305500',
  '48548.12497460',
  '0'],
 [1502964000000,
  '4441.10000000',
  '4485.39000000',
  '4399.81000000',
  '4427.30000000',
  '51.88345200',
  1502971199999,
  '231042.29822033',
  177,
  '47.71457800',
  '212505.92617935',
  '0'],
 [1502971200000,
  '4436.06000000',
  '4459.00000000',
  '4411.00000000',
  '4459.00000000',
  '54.34483400',
  1502978399999,
  '241103.15802063',
  226,
  '41.87703600',
  '185858.30968151',
  '0'],
 [